# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 60 new papers today
          16 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/15 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.03348


extracting tarball to tmp_2209.03348...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.03364


extracting tarball to tmp_2209.03364...

...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.03886


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.03809/template.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2209.03886...

 done.


/tmp/ipykernel_2191/4030337529.py:34: LatexWarning: 2209.03886 did not run properly
[Errno 2] No such file or directory: 'gs'
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03367-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03367) | **Discovery of Line Pressure Broadening and Direct Constraint on Gas  Surface Density in a Protoplanetary Disk**  |
|| Tomohiro C. Yoshida, et al. -- incl., <mark>Takahiro Ueda</mark> |
|*Appeared on*| *2022-09-09*|
|*Comments*| *12 pages, 5 figures, accepted for publication in ApJL*|
|**Abstract**| The gas surface density profile of protoplanetary disks is one of the most fundamental physical properties to understand planet formation. However, it is challenging to determine the surface density profile observationally, because the H$_2$ emission cannot be observed in low-temperature regions. We analyzed the Atacama Large Millimeter/submillimeter Array (ALMA) archival data of the \co line toward the protoplanetary disk around TW Hya and discovered extremely broad line wings due to the pressure broadening. In conjunction with a previously reported optically thin CO isotopologue line, the pressure broadened line wings enabled us to directly determine the midplane gas density for the first time. The gas surface density at $\sim5$ au from the central star reaches $\sim 10^3\ {\rm g\ cm^{-2}}$, which suggests that the inner region of the disk has enough mass to form a Jupiter-mass planet. Additionally, the gas surface density drops at the inner cavity by $\sim2$ orders of magnitude compared to outside the cavity. We also found a low CO abundance of $\sim 10^{-6}$ with respect to H$_2$, even inside the CO snowline, which suggests conversion of CO to less volatile species. Combining our results with previous studies, the gas surface density jumps at $r\sim 20$ au, suggesting that the inner region ($3<r<20$ au) might be the magnetorotational instability dead zone. This study sheds light on direct gas-surface-density constraint without assuming the CO/H$_2$ ratio using ALMA. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03585-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03585) | **A Spectroscopic Road Map for Cosmic Frontier: DESI, DESI-II, Stage-5**  |
|| David J. Schlegel, et al. -- incl., <mark>Joseph DeRose</mark>, <mark>Daniel Green</mark>, <mark>Dustin Lang</mark> |
|*Appeared on*| *2022-09-09*|
|*Comments*| *Contribution to Snowmass 2021*|
|**Abstract**| In this white paper, we present an experimental road map for spectroscopic experiments beyond DESI. DESI will be a transformative cosmological survey in the 2020s, mapping 40 million galaxies and quasars and capturing a significant fraction of the available linear modes up to z=1.2. DESI-II will pilot observations of galaxies both at much higher densities and extending to higher redshifts. A Stage-5 experiment would build out those high-density and high-redshift observations, mapping hundreds of millions of stars and galaxies in three dimensions, to address the problems of inflation, dark energy, light relativistic species, and dark matter. These spectroscopic data will also complement the next generation of weak lensing, line intensity mapping and CMB experiments and allow them to reach their full potential. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03348-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03348) | **The JWST Resolved Stellar Populations Early Release Science Program I.:  NIRCam Flux Calibration**  |
|| Martha L. Boyer, et al. -- incl., <mark>Benjamin F. Williams</mark> |
|*Appeared on*| *2022-09-09*|
|*Comments*| *Accepted for publication in RNAAS*|
|**Abstract**| We use globular cluster data from the Resolved Stellar Populations Early Release Science (ERS) program to validate the flux calibration for the Near Infrared Camera (NIRCam) on the James Webb Space Telescope (JWST). We find a significant flux offset between the eight short wavelength detectors, ranging from 1-23% (about 0.01-0.2 mag) that affects all NIRCam imaging observations. We deliver improved zeropoints for the ERS filters and show that alternate zeropoints derived by the community also improve the calibration significantly. We also find that the detector offsets appear to be time variable by up to at least 0.1 mag. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03459-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03459) | **A Bayesian Calibration Framework for EDGES**  |
|| Steven G. Murray, et al. -- incl., <mark>Nivedita Mahesh</mark> |
|*Appeared on*| *2022-09-09*|
|*Comments*| *18 pages + 3 for appendices. 13 figures. Accepted to MNRAS*|
|**Abstract**| We develop a Bayesian model that jointly constrains receiver calibration, foregrounds and cosmic 21cm signal for the EDGES global 21\,cm experiment. This model simultaneously describes calibration data taken in the lab along with sky-data taken with the EDGES low-band antenna. We apply our model to the same data (both sky and calibration) used to report evidence for the first star formation in 2018. We find that receiver calibration does not contribute a significant uncertainty to the inferred cosmic signal (<1%), though our joint model is able to more robustly estimate the cosmic signal for foreground models that are otherwise too inflexible to describe the sky data. We identify the presence of a significant systematic in the calibration data, which is largely avoided in our analysis, but must be examined more closely in future work. Our likelihood provides a foundation for future analyses in which other instrumental systematics, such as beam corrections and reflection parameters, may be added in a modular manner. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03502-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03502) | **Non-detection of He I in the atmosphere of GJ1214b with Keck/NIRSPEC, at  a time of minimal telluric contamination**  |
|| Jessica J. Spake, et al. -- incl., <mark>Heather A. Knutson</mark>, <mark>Michael Zhang</mark> |
|*Appeared on*| *2022-09-09*|
|*Comments*| *Accepted for publication in ApJL*|
|**Abstract**| Observations of helium in exoplanet atmospheres may reveal the presence of large gaseous envelopes, and indicate ongoing atmospheric escape. Orell-Miquel et al. (2022) used CARMENES to measure a tentative detection of helium for the sub-Neptune GJ 1214b, with a peak excess absorption reaching over 2% in transit depth at 10830 Angstroms. However, several non-detections of helium had previously been reported for GJ 1214b. One explanation for the discrepancy was contamination of the planetary signal by overlapping telluric absorption- and emission lines. We used Keck/NIRSPEC to observe another transit of GJ 1214b at 10830 Angstroms, at a time of minimal contamination by telluric lines, and did not observe planetary helium absorption. Accounting for correlated noise in our measurement, we place an upper limit on the excess absorption size of 1.22% (95% confidence). We find that the discrepancy between the CARMENES and NIRSPEC observations is unlikely to be caused by using different instruments or stellar activity. It is currently unclear whether the difference is due to correlated noise in the observations, or variability in the planetary atmosphere. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03545-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03545) | **Inflationary magnetogenesis of primordial magnetic fields with multiple  vector fields**  |
|| Yu Li, <mark>Le-Yao Zhang</mark> |
|*Appeared on*| *2022-09-09*|
|*Comments*| *16 pages, 0 figure,submitted to JCAP*|
|**Abstract**| In this paper, we discussed the multiple vector fields during the inflation era and the inflationary magnetogenesis with multiple vector fields. Instead of a single coupling function in single vector field models, the coupling matrix between vector fields and scalar field which drive the inflation is introduced. The dynamical equations for multiple vector fields are obtained and applied to the inflation era. We discussed three cases for the double-field model. In no mutual-coupling case, one can find that both electric and magnetic spectrum can be scale-invariant at the end of inflation, meanwhile, the strong coupling problem can be avoided. The effect of mutual-coupling between different vector fields is also discussed. We found that weak mutual-coupling can lead to the slightly blue spectrum of the magnetic field. On the other hand, in the strong mutual-coupling case, the scale-invariant magnetic spectrum can also be obtained but the energy density of electromagnetic fields either lead to the backreaction problem or is diluted by inflation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03809-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03809) | **A PeVatron Candidate: Modelling the Boomerang Nebula in X-ray Band**  |
|| <mark>Xuan-Han Liang</mark>, et al. |
|*Appeared on*| *2022-09-09*|
|*Comments*| *15 pages, 5 figures, 5 tables; Invited contribution to Special Issue of Universe "Advances in Astrophysics and Cosmology in China - in Memory of Prof. Tan Lu"*|
|**Abstract**| Pulsar wind nebula (PWN) Boomerang and the associated supernova remnant (SNR) G106.3+2.7 are among candidates for the ultra-high-energy (UHE) gamma-ray counterparts published by LHAASO. Although the centroid of the extended source, LHAASO J2226+6057, deviates from the pulsar's position by about $0.3^\circ$, the source partially covers the PWN. Therefore, we cannot totally exclude the possibility that a part of the UHE emission comes from the PWN. Previous studies mainly focus on whether the SNR is a PeVatron, while neglecting the energetic PWN. Here, we explore the possibility of the Boomerang Nebula being a PeVatron candidate by studying its X-ray radiation. By modelling the diffusion of relativistic electrons injected in the PWN, we fit the radial profiles of the X-ray surface brightness and the photon index. The solution with a magnetic field $B=140\mu$G can well reproduce the observed profiles and implies a severe suppression of IC scattering of electrons. Therefore, a proton component need be introduced to account for the UHE emission, in light of recent LHAASO's measurement on Crab Nebula, if future observations reveal part of the UHE emission originating from the PWN. In this sense, Boomerang Nebula would be a hadronic PeVatron. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.03886-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.03886) | **Mass Production of 2021 KMTNet Microlensing Planets III: Analysis of  Three Giant Planets**  |
|| In-Gu Shin, et al. -- incl., <mark>Andrew Gould</mark>, <mark>Seung-Lee Kim</mark>, <mark>Chung-Uk Lee</mark>, <mark>Dong-Joo Lee</mark>, <mark>Yongseok Lee</mark> |
|*Appeared on*| *2022-09-09*|
|*Comments*| *16 pages, 11 figures, 7 tables. Submitted to the AAS journal*|
|**Abstract**| We present the analysis of three more planets from the KMTNet 2021 microlensing season. KMT-2021-BLG-0119Lb is a $\sim 6\, M_{\rm Jup}$ planet orbiting an early M-dwarf or a K-dwarf, KMT-2021-BLG-0192Lb is a $\sim 2\, M_{\rm Nep}$ planet orbiting an M-dwarf, and KMT-2021-BLG-0192Lb is a $\sim 1.25\, M_{\rm Nep}$ planet orbiting a very--low-mass M dwarf or a brown dwarf. These by-eye planet detections provide an important comparison sample to the sample selected with the AnomalyFinder algorithm, and in particular, KMT-2021-BLG-2294, is a case of a planet detected by-eye but not by-algorithm. KMT-2021-BLG-2294Lb is part of a population of microlensing planets around very-low-mass host stars that spans the full range of planet masses, in contrast to the planet population at $\lesssim 0.1\, $ au, which shows a strong preference for small planets. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error [Errno 2] No such file or directory: 'gs'</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.03367.md
    + _build/html/tmp_2209.03367/./vis_mom_vc.png
    + _build/html/tmp_2209.03367/./spectra.png
    + _build/html/tmp_2209.03367/./radprof.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\co}{{\rm ^{12}CO} J=3-2 }$
$\newcommand{\kms}{{\rm km s^{-1}}}$
$\newcommand{\mJB}{{\rm mJy {beam}^{-1}}}$
$\newcommand{\gcmt}{{\rm g {cm}^{-2}}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{$\co$}{{\rm ^{12}CO} J=3-2 }$
$\newcommand{$\kms$}{{\rm km s^{-1}}}$
$\newcommand{$\mJB$}{{\rm mJy {beam}^{-1}}}$
$\newcommand{$\gcmt$}{{\rm g {cm}^{-2}}}$</div>



<div id="title">

# Discovery of Line Pressure Broadening and Direct Constraint on Gas Surface Density in a Protoplanetary Disk

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.03367-b31b1b.svg)](https://arxiv.org/abs/2209.03367)<mark>Appeared on: 2022-09-09</mark> - _12 pages, 5 figures, accepted for publication in ApJL_

</div>
<div id="authors">

Tomohiro C. Yoshida, et al. -- incl., <mark>Takahiro Ueda</mark>

</div>
<div id="abstract">

**Abstract:** The gas surface density profile of protoplanetary disks is one of the most fundamental physical properties to understanding planet formation.However, it is challenging to determine the surface dunsity profile observationally, because the H$_2$emission cannot be observed in low-temperature regions.We analyzed the Atacama Large Millimeter/submillimeter Array (ALMA) archival data of the$\co$line toward the protoplanetary disk around TW Hya, and discovered extremely broad line wings due to the pressure broadening.In conjunction with a previously reported optically thin CO isotopologue line, the pressure broadened line wings enabled us to directly determine the midplane gas density for the first time.The gas surface density at$\sim5$au from the{ central star}reaches$\sim 10^3 {\rm g cm^{-2}}$, which suggests that the{ inner region of the}disk has enough mass to form a Jupiter-mass planet.{ Additionally, the gas surface density drops at the inner cavity by $\sim2$ orders of magnitude compared to outside the cavity.}We also found a low CO abundance of$\sim 10^{-6}$with respect to H$_2$, even inside the CO snowline, which suggests conversion of CO to less volatile species.Combining{ our results with}previous studies, the gas surface density jumps at$r\sim 20$au, suggesting that the inner region ($3<r<20$au) might be the magnetorotational instability dead zone.This study sheds light on direct gas-surface-density constraint without assuming the CO/H$_2$ratio using ALMA.

</div>

<div id="div_fig1">

<img src="tmp_2209.03367/./vis_mom_vc.png" alt="Fig2" width="100%"/>

**Figure 2. -** (a) uv-distance vs. azimuthally averaged complex visibilities (black circles). The green points and gray dotted lines indicate the best-fitted model and the cavity-sized uniform disk model, respectively.
    (b) Averaged intensity maps with Briggs weighting (robust = 0) for the red- and blue-shifted line wings in the color scale and the black solid contour with 3 $\sigma$ and 5 $\sigma$($\sigma = 0.38 $\mJB$$). The maps with natural weighting are shown in Figure \ref{fig:comparison}.
    (c) 233 GHz continuum maps of \citet{tsuk19}.
    (d) Centroid velocity map.
    The white dashed circles in panel (b), (c), and (d) indicate the inner cavity radius \citep{andr16}. The gray solid contour shows the dust continuum emission \citep{tsuk19}. The contour starts from the brightness temperature of 10 K with an interval of 5 K.
    \label{fig:all} (*fig:all*)

</div>
<div id="div_fig2">

<img src="tmp_2209.03367/./spectra.png" alt="Fig1" width="100%"/>

**Figure 1. -**  Integrated spectrum of the $\co$ line in the $0$\farcs$ 2$ aperture (black solid line) with the uncertainty (gray shaded area) and a $3\sigma$ noise level (black dotted line).
    The gray dashed line indicates synthetic spectra of the \citet{huan18} model with a Gaussian line profile, while the green solid line shows the best-fit model results using the Voigt line profile (see Section \ref{sec:mod}).
    Blue- and red-masked ranges are used to create the averaged visibilities for each wing.
    $v$ denotes the velocity shift from the systemic velocity.
    \label{fig:spec} (*fig:spec*)

</div>
<div id="div_fig3">

<img src="tmp_2209.03367/./radprof.png" alt="Fig5" width="100%"/>

**Figure 5. -** { (a) Radial profiles of the gas surface density. The green solid lines show the best-fit model. The models with parameters randomly selected from the posterior distributions are plotted in light green. The MMSN model and the \citet{cala21} model are also shown in gray lines.
    The gray dotted lines indicate the gas surface density converted from the dust surface density of \citet{maci21}, assuming a gas-to-dust surface density ratio of 10 and 100. In the right panel, we also show the gas surface density when the Toomre Q equals unity, and the gas surface density profiles starting from $r=10$ au of the best-fit profile with fixing the total gas masses to 0.023 $M_\odot$ or 0.06 $M_\odot$.
    (b) The $\alpha$ viscosity parameter for each gas surface density profile, assuming a steady accretion disk. }\label{fig:radprof} (*fig:radprof*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

29  publications in the last 7 days.
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers